# Tracking EPA's COVID-19 non-enforcement policy
In late March 2020, EPA released a memo announcing that it would not penalize regulated industries that fail to meet their monitoring and reporting requirements due to COVID-19. Specifically EPA has said that it:

"is not seeking penalties for noncompliance only in circumstances that involve routine monitoring and reporting requirements, if, on a case-by-case basis, EPA agrees that such noncompliance was caused by the COVID-19 pandemic."

This may have a number of public and environmental health impacts. See our response to EPA's memo here: https://envirodatagov.org/epas-covid-19-leniency-is-a-free-pass-to-pollute/

What is the effect of EPA's non-enforcement of Clean Air Act, Clean Water Act, and hazardous waste permits? Using this notebook, you can track how facilities' monitoring and reporting of air, water, and waste hazards has changed in your state over the past few months, compared to previous years. 

---

## Setup
Here we load some helper code to get us going.

In [ ]:
# Import libraries
import urllib.parse
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import requests
import csv
import datetime
import folium

## Select your state
By typgin in its postal code abbreviation

In [ ]:
my_state="WI" # Could switch to dropdown menu

## Water
First, we'll look at how facilities regulated under the Clean Water Act have altered their required monitoring practices.

Run this code, which will query our copy of the ECHO database and pull information on regulated facilities in your state.

Specifically, we'll find records of facilities out of compliance - that is, violating their permits - due to "Non-Receipt of DMR/Schedule Report" DMR stands for Discharge Monitoring Reports, and are required by the CWA's National Pollutant Discharge Elimination System (NPDES). 

Not submitting these reports on schedule can lead to "Reportable Non-Compliance" with NPDES and CWA. According to the EPA, "DMR values not received within 31 days of the DMR form due date result in the generation of a violation code (D80 or D90). ICIS-NPDES identifies these DMR non-receipt violations and automatically creates violation codes for the missing DMR values with monitoring requirements (D80) and missing DMR values with effluent limits (D90). EPA's data sharing policy allows states a 40-day window to report DMR values to EPA's data system; therefore, DMR values reported on time to state agencies and shared with EPA within 40 days do not contribute to permit level noncompliance status."

In this case, "N" does NOT mean no - it just is a code for the kind of violation event (non-reporting).

In [ ]:
sql = "select NPDES_ID, SCHEDULE_DATE, RNC_DETECTION_CODE" + \
    " from NPDES_PS_VIOLATIONS where RNC_DETECTION_CODE = 'N' and " \
    " NPDES_ID like '" + my_state + "%'"
url='http://apps.tlt.stonybrook.edu/echoepa/?query='
data_location=url+urllib.parse.quote(sql)
#

data = pd.read_csv(data_location,header = 0)
data.set_index("NPDES_ID", inplace=True)
data

### Let's organize this information by date
We're curious to track whether non-receipt of DMRs has increased due to COVID, so we have to be able to summarize and order facilities' violations across time.

In [ ]:
# The NPDES_IDS in ECHO_EXPORTER can contain multiple ids for a facility. 
data['SCHEDULE_DATE'] = pd.to_datetime(data['SCHEDULE_DATE'], format="%m/%d/%Y")
by_date = data.groupby('SCHEDULE_DATE')[['RNC_DETECTION_CODE']].count()
by_date=by_date.resample('M').sum() # Summarize by month. Or Y for year?
by_date=by_date.loc['2017-01-01':'2020-05-01'] #filter to relatively recent
by_date

### Plot this ^ !!!
It's all well and good to have this table, but it's hard to pick out patterns from tabular data. Let's plot it as what's called a histogram in order to see what's going on.

In [ ]:
#fig, ax = plt.subplots(figsize=(15,15))
#my_cd_groups.plot(ax=ax)

chart_title = "Total CWA Non-Compliance due to Missing or Late Reports"
ax = by_date.plot(kind='bar', title = chart_title, figsize=(20, 10), legend=False, fontsize=16)
ax.set_xlabel("Month", fontsize=16)
ax.set_ylabel("Count", fontsize=16)

### Which facilities didn't report in the month of April?
We don't have that information yet, but it'll be a good indicator of the impact of EPA's memo, which went into effect that month.

So, for now, let's map those that didn't report in March 2020.

In [ ]:
latest = data[(data["SCHEDULE_DATE"] >= '2020-03-01') & (data["SCHEDULE_DATE"] < '2020-04-01')] #filter to March

if (len(latest.index)>0):
    #get facility information from ECHO
    sql = "select FAC_NAME, NPDES_IDS, FAC_LAT, FAC_LONG" + \
        " from ECHO_EXPORTER " + \
        " where NPDES_FLAG = 'Y' and FAC_STATE = '" + my_state + "'"
    url='http://apps.tlt.stonybrook.edu/echoepa/?query='
    data_location=url+urllib.parse.quote(sql)
    echo_data = pd.read_csv(data_location,encoding='iso-8859-1',header = 0)
    echo_data.set_index( 'NPDES_IDS', inplace=True )

    #merge echo and npdes data
    latest = latest.join(echo_data)

    def mapper(df):
        # Initialize the map
        m = folium.Map(
            location = [df.mean()["FAC_LAT"], df.mean()["FAC_LONG"]],
            zoom_start = 11
        )

        # Add a clickable marker for each facility
        for index, row in df.iterrows():
            folium.Marker(
                location = [row["FAC_LAT"], row["FAC_LONG"]],
                popup = row["FAC_NAME"] ).add_to(m)

        # Show the map
        return m

    map_of_facilities = mapper(latest)
    map_of_facilities
else:
    print("Actually, there were no reporting violations for March in "+my_state)
    

## Air
Now, let's look at violations of the Clean Air Act's reporting requirements. If facilities are not reporting their emissions, they may be releasing more pollutants, which can actually contribute to COVID-19 outcomes.


## Waste
Now, let's look at violations of the Resource Conservation and Recovery Act's (RCRA) requirements.